In [1]:
def CountDuplicates(S):
## CountDuplicates is used to count the number of connected half-edges
## in a graph represented by the set partition S. S is a list of lists, 
## and we count the number of times any number appears more than once in S
    counts = {}
    count = 0
    for s in S:
        for e in s:
            counts[e] = 1 if e in counts else 0
    return sum(counts.values())

def DFactorialk(k):
## The falling factorial D...(D-k+1)
    D = var("D")
    return prod([D-i for i in [0..(k-1)]])

def keepElements(L, elms):
## The two point function can be seen as a map from a set partition L of a set of four elements,
## into a list of lists of the form {L', X}, where L' is related to L by keeping only the elements
## in L which are also in X. We implement the restriction of L to X using L' = keepElements(L, X).
    l = []
    elms = flatten(elms)
    if isinstance(L,list):
        for part in L:
            l.append(flatten(keepElements(part, elms)))
    elif L in elms:
        l.append(L)
    return l

def diagramprod(g1,g2):
## For n-points, which through wick contractions involve products of
## 1 and 2-points we need a product of diagrams. This is an algebra structure
## where elements are linear combinations of diagrams. The product is multiplication of
## the coefficients and concatenation of the lists. diagramprod(g1,g2) takes two lists [coefficient, list of lists]
## and returns [coefficient1*coefficient2, [lists1, lists2]]
    return [g1[0]*g2[0], g1[1]+g2[1]]

def algprod(G1, G2):
## This function implements the product for two linear combinations of graphs: G1, G2. G1, G2 are lists of the form
## [coefficient, list of lists]. algprod returns the list where diagramprod is distributed over G1 and G2.
    return [diagramprod(g1,g2) for g1 in G1 for g2 in G2]
 
def recurseprod(G):
## Implements the product of an arbitrary number of G = [G1,G2,G3,...]. This is done recursively. I.e
## recurseprod([G1, G2, G3,..]) = algprod(G1, recurseprod([G2,G3,...]), etc.
    if len(G) > 1:
        return algprod(G[0],recurseprod(G[1:]))
    else:
        return G[0]                  

def GraphPolynomialData(V):
## Given a list of vertices, where each vertex is a list of legs (numbered using integers)
## this produces a list of data necessary to reproduce the corresponding polynomial.
## GraphPolynomialData(V) returns a list [l, l3], where l3 is the total number of edges in V
## l is a list of pairs [l1,l2], where l1 is the number of parts in a partition
## and l2 is the number of connected edges in the partition.
    VertexPartitions = SetPartitions(len(V)).list()
    l = []
    l3 = 0
    for partition in VertexPartitions:     
        l1 = len(partition)
        l2 = 0
        for part in partition:
            l2 = l2 + CountDuplicates([V[i-1] for i in part])  
        l.append([l1, l2])
    if len(flatten(V)) > 0:
        l3 = CountDuplicates(V)
    return [l, l3]

def GraphPolynomialFromData(Data):
## This function takes the data returned from GraphPolynomialData and produces the corresponding graph function
    D = var("D")        
    polynomial = 0
    for d in Data[0]:
        polynomial = polynomial + (DFactorialk(d[0]))*(1-D)^d[1]
    return polynomial/(-D)^Data[1]

def GP(G):
## GP takes a list of diagrams with coefficients and produces the sum of polynomials, weighted by the coefficients.
## This is used to complete n-point calculations, after having done the multiplication of diagrams, coming from
## wick contractions.
    p = 0
    for g in G:
        if 1 not in [len(part) for part in g[1]]:
            p = p + g[0]*GraphPolynomialFromData(GraphPolynomialData(g[1]))
    return p

def OnePointM(M):
    D = var('D')
    rho1 = var('rho_1')
    rho2 = var('rho_2')
    return [
            [rho1/D, []],
            [rho2/sqrt(D-1), [[M[0],M[1]]]]
           ]

def OnePointN(N):
    D = var('D')
    rho3 = var('rho_3')
    rho4 = var('rho_4')
    return [
            [rho3/D, []],
            [rho4/sqrt(D-1), [[N[0],N[1]]]]
           ]
    
def TwoPointMN(M,N,i):
    D = var('D')
    XY0_11 = var('XY0_11', latex_name=r"({\Lambda}^{-1}_{V_0})^{XY}_{11}")
    XY0_22 = var('XY0_22', latex_name=r"({\Lambda}^{-1}_{V_0})^{XY}_{22}")
    XY0_12 = var('XY0_12', latex_name=r"({\Lambda}^{-1}_{V_0})^{XY}_{12}")
    XY0_21 = var('XY0_21', latex_name=r"({\Lambda}^{-1}_{V_0})^{XY}_{21}")
    XYH_11 = var('XYH_11', latex_name=r"({\Lambda}^{-1}_{V_H})^{XY}_{11}")
    XYH_22 = var('XYH_22', latex_name=r"({\Lambda}^{-1}_{V_H})^{XY}_{22}")
    XYH_33 = var('XYH_33', latex_name=r"({\Lambda}^{-1}_{V_H})^{XY}_{33}")
    XYH_12 = var('XYH_12', latex_name=r"({\Lambda}^{-1}_{V_H})^{XY}_{12}")
    XYH_21 = var('XYH_21', latex_name=r"({\Lambda}^{-1}_{V_H})^{XY}_{21}")
    XYH_13 = var('XYH_13', latex_name=r"({\Lambda}^{-1}_{V_H})^{XY}_{13}")
    XYH_31 = var('XYH_31', latex_name=r"({\Lambda}^{-1}_{V_H})^{XY}_{31}")
    XYH_23 = var('XYH_23', latex_name=r"({\Lambda}^{-1}_{V_H})^{XY}_{23}")
    XYH_32 = var('XYH_32', latex_name=r"({\Lambda}^{-1}_{V_H})^{XY}_{32}")
    XYV2 = var('XYV2', latex_name=r"({\Lambda}^{-1}_{V_2})^{XY}")
    XYV3 = var('XYV3', latex_name=r"({\Lambda}^{-1}_{V_3})^{XY}")        
    G = [
            [XY0_11/D^2, []],
            [XY0_22/(D-1), [[M[0], M[1]],[N[0], N[1]]]],
            [XY0_12/(D*sqrt(D-1)), [[N[0], N[1]]]],
            [XY0_21/(D*sqrt(D-1)), [[M[0], M[1]]]],
            [XYH_11/D, [[M[1],N[1]]]],
            [XYH_22/D, [[M[0],N[0]]]],
            [D*XYH_33/(D-2), [[M[0],M[1],i],[N[0],N[1],i]]],
            [XYH_12/D, [[M[1],N[0]]]],
            [XYH_21/D, [[M[0],N[1]]]],
            [XYH_13/sqrt(D-2), [[M[1],N[0],N[1]]]],
            [XYH_31/sqrt(D-2), [[M[0],M[1],N[1]]]],
            [XYH_23/sqrt(D-2), [[M[0],N[0],N[1]]]],
            [XYH_32/sqrt(D-2), [[M[0],M[1],N[0]]]],
            [XYV2/2, [[M[0],N[0]],[M[1],N[1]]]],
            [XYV2/2, [[M[0],N[1]],[M[1],N[0]]]],
            [-D*XYV2/(D-2), [[M[0],M[1],i],[N[0],N[1],i]]],
            [-XYV2/(D-1), [[M[0],M[1]],[N[0],N[1]]]],
            [+XYV3/2, [[M[0],N[0]],[M[1],N[1]]]],
            [-XYV3/2, [[M[0],N[1]],[M[1],N[0]]]]
    ]
    return G

def TwoPointMM(M,N,i):
    D = var('D')
    X0_11 = var('X0_11', latex_name=r"({\Lambda}^{-1}_{V_0})^{X}_{11}")
    X0_22 = var('X0_22', latex_name=r"({\Lambda}^{-1}_{V_0})^{X}_{22}")
    X0_12 = var('X0_12', latex_name=r"({\Lambda}^{-1}_{V_0})^{X}_{12}")
    X0_21 = var('X0_21', latex_name=r"({\Lambda}^{-1}_{V_0})^{X}_{21}")
    XH_11 = var('XH_11', latex_name=r"({\Lambda}^{-1}_{V_H})^{X}_{11}")
    XH_22 = var('XH_22', latex_name=r"({\Lambda}^{-1}_{V_H})^{X}_{22}")
    XH_33 = var('XH_33', latex_name=r"({\Lambda}^{-1}_{V_H})^{X}_{33}")
    XH_12 = var('XH_12', latex_name=r"({\Lambda}^{-1}_{V_H})^{X}_{12}")
    XH_21 = var('XH_21', latex_name=r"({\Lambda}^{-1}_{V_H})^{X}_{21}")
    XH_13 = var('XH_13', latex_name=r"({\Lambda}^{-1}_{V_H})^{X}_{13}")
    XH_31 = var('XH_31', latex_name=r"({\Lambda}^{-1}_{V_H})^{X}_{31}")
    XH_23 = var('XH_23', latex_name=r"({\Lambda}^{-1}_{V_H})^{X}_{23}")
    XH_32 = var('XH_32', latex_name=r"({\Lambda}^{-1}_{V_H})^{X}_{32}")
    XV2 = var('XV2', latex_name=r"({\Lambda}^{-1}_{V_2})^{X}")
    XV3 = var('XV3', latex_name=r"({\Lambda}^{-1}_{V_3})^{X}")       
    G = [
            [X0_11/D^2, []],
            [X0_22/(D-1), [[M[0], M[1]],[N[0], N[1]]]],
            [X0_12/(D*sqrt(D-1)), [[N[0], N[1]]]],
            [X0_12/(D*sqrt(D-1)), [[M[0], M[1]]]],
            [XH_11/D, [[M[1],N[1]]]],
            [XH_22/D, [[M[0],N[0]]]],
            [D*XH_33/(D-2), [[M[0],M[1],i],[N[0],N[1],i]]],
            [XH_12/D, [[M[1],N[0]]]],
            [XH_12/D, [[M[0],N[1]]]],
            [XH_13/sqrt(D-2), [[M[1],N[0],N[1]]]],
            [XH_13/sqrt(D-2), [[M[0],M[1],N[1]]]],
            [XH_23/sqrt(D-2), [[M[0],N[0],N[1]]]],
            [XH_23/sqrt(D-2), [[M[0],M[1],N[0]]]],
            [XV2/2, [[M[0],N[0]],[M[1],N[1]]]],
            [XV2/2, [[M[0],N[1]],[M[1],N[0]]]],
            [-D*XV2/(D-2), [[M[0],M[1],i],[N[0],N[1],i]]],
            [-XV2/(D-1), [[M[0],M[1]],[N[0],N[1]]]],
            [+XV3/2, [[M[0],N[0]],[M[1],N[1]]]],
            [-XV3/2, [[M[0],N[1]],[M[1],N[0]]]]
    ]
    return G 

def TwoPointNN(M,N,i):
    D = var('D')
    Y0_11 = var('Y0_11', latex_name=r"({\Lambda}^{-1}_{V_0})^{Y}_{11}")
    Y0_22 = var('Y0_22', latex_name=r"({\Lambda}^{-1}_{V_0})^{Y}_{22}")
    Y0_12 = var('Y0_12', latex_name=r"({\Lambda}^{-1}_{V_0})^{Y}_{12}")
    Y0_21 = var('Y0_21', latex_name=r"({\Lambda}^{-1}_{V_0})^{Y}_{21}")
    YH_11 = var('YH_11', latex_name=r"({\Lambda}^{-1}_{V_H})^{Y}_{11}")
    YH_22 = var('YH_22', latex_name=r"({\Lambda}^{-1}_{V_H})^{Y}_{22}")
    YH_33 = var('YH_33', latex_name=r"({\Lambda}^{-1}_{V_H})^{Y}_{33}")
    YH_12 = var('YH_12', latex_name=r"({\Lambda}^{-1}_{V_H})^{Y}_{12}")
    YH_21 = var('YH_21', latex_name=r"({\Lambda}^{-1}_{V_H})^{Y}_{21}")
    YH_13 = var('YH_13', latex_name=r"({\Lambda}^{-1}_{V_H})^{Y}_{13}")
    YH_31 = var('YH_31', latex_name=r"({\Lambda}^{-1}_{V_H})^{Y}_{31}")
    YH_23 = var('YH_23', latex_name=r"({\Lambda}^{-1}_{V_H})^{Y}_{23}")
    YH_32 = var('YH_32', latex_name=r"({\Lambda}^{-1}_{V_H})^{Y}_{32}")
    YV2 = var('YV2', latex_name=r"({\Lambda}^{-1}_{V_2})^{X}")
    YV3 = var('YV3', latex_name=r"({\Lambda}^{-1}_{V_3})^{Y}")    
    G = [
            [Y0_11/D^2, []],
            [Y0_22/(D-1), [[M[0], M[1]],[N[0], N[1]]]],
            [Y0_12/(D*sqrt(D-1)), [[N[0], N[1]]]],
            [Y0_12/(D*sqrt(D-1)), [[M[0], M[1]]]],
            [YH_11/D, [[M[1],N[1]]]],
            [YH_22/D, [[M[0],N[0]]]],
            [D*YH_33/(D-2), [[M[0],M[1],i],[N[0],N[1],i]]],
            [YH_12/D, [[M[1],N[0]]]],
            [YH_12/D, [[M[0],N[1]]]],
            [YH_13/sqrt(D-2), [[M[1],N[0],N[1]]]],
            [YH_13/sqrt(D-2), [[M[0],M[1],N[1]]]],
            [YH_23/sqrt(D-2), [[M[0],N[0],N[1]]]],
            [YH_23/sqrt(D-2), [[M[0],M[1],N[0]]]],
            [YV2/2, [[M[0],N[0]],[M[1],N[1]]]],
            [YV2/2, [[M[0],N[1]],[M[1],N[0]]]],
            [-D*YV2/(D-2), [[M[0],M[1],i],[N[0],N[1],i]]],
            [-YV2/(D-1), [[M[0],M[1]],[N[0],N[1]]]],
            [+YV3/2, [[M[0],N[0]],[M[1],N[1]]]],
            [-YV3/2, [[M[0],N[1]],[M[1],N[0]]]]
    ]
    return G 
    
def Npoint(Mtuples, Ntuples, invariant):
## The free field N-point correlator is a sum over all possible partitions of the 
## N elementary fields where the partition has parts of size one or two.
## Fields in the same part are put into a one or two point correlator respectively, and the
## product of these is calculated.
## Npoint takes a list Mtuples of pairs [integer1, integer2] which label the indices of the M-matrices,
## and a second list Ntuples for the N-matrices.
## invariant is a set partition(list of lists) of the integers in Mtuples and Ntuples, describing
## a particular permutation invariant observable.
## Integers in the same part in invariant correspond to a sum where the corresponding indices are
## set equal.
    contractions = []
    Tuples = Mtuples+Ntuples
    m = len(Mtuples)
    n = len(Ntuples)
    N = (m+n)
    wicks = [s for s in SetPartitions(range(N)) if all(x<3 for x in s.shape())]
    for wick in wicks:
        twopts = []
        onepts = []
        i = 2*(m+n)
        for matrix1, *matrix2 in wick:  
            if not matrix2:         
                ## matrix2 returns false if empty, i.e. if w has just one element (one point)
                onepts += [OnePointM(Tuples[matrix1]) if matrix1 < m else OnePointN(Tuples[matrix1])]
            else:           
                i += 1
                if matrix1 < m and matrix2[0] < m: ## <MM>
                    twopts += [TwoPointMM(Tuples[matrix1],Tuples[matrix2[0]],i)]
                elif matrix1 < m and matrix2[0] >= m: ##  <MN>
                    twopts += [TwoPointMN(Tuples[matrix1],Tuples[matrix2[0]],i)]
                elif matrix1 >= m and matrix2[0] < m: ##  <NM>
                    twopts += [TwoPointMN(Tuples[matrix2[0]],Tuples[matrix1],i)]
                elif matrix1 >= m and matrix2[0] >= m: ## <NN>
                    twopts += [TwoPointNN(Tuples[matrix1],Tuples[matrix2[0]],i)]
        contractions += recurseprod(twopts+onepts)
    G = [[g[0], keepElements(invariant, g[1])+g[1]] for g in contractions]
    return GP(G)

In [ ]:
## Cubic expectation values

In [ ]:
## Expectation value <M_ii M_ii N_ii>
Npoint([[1,2],[3,4]],[[5,6]],[[1,2,3,4,5,6]]).full_simplify()

In [12]:
## Expectation value <M_ij M_ij N_ij>
Npoint([[1,2],[3,4]],[[5,6]],[[1,3,5],[2,4,6]]).full_simplify()

1/2*(2*(2*(D - 1)*XY0_11 + 2*(D - 1)*XY0_22 + 2*(D^2 - 2*D + 1)*XYH_11 + 2*(D^2 - 2*D + 1)*XYH_22 + 2*(D^2 - 2*D + 1)*XYH_33 + (D^3 - 4*D^2 + 3*D)*XYV2 + (D^3 - 4*D^2 + 5*D - 2)*XYV3)*rho_1 + 4*((D - 1)*XY0_12 + (D - 1)*XY0_21)*rho_2 + (2*(D - 1)*rho_1^2 + 2*(D - 1)*rho_2^2 + 2*(D - 1)*X0_11 + 2*(D - 1)*X0_22 + 2*(D^2 - 2*D + 1)*XH_11 + 2*(D^2 - 2*D + 1)*XH_22 + 2*(D^2 - 2*D + 1)*XH_33 + (D^3 - 4*D^2 + 3*D)*XV2 + (D^3 - 4*D^2 + 5*D - 2)*XV3)*rho_3 + 4*((D - 1)*rho_1*rho_2 + (D - 1)*X0_12)*rho_4 + (2*(2*(D - 2)*XY0_22 + 2*(D - 1)*XYH_12 + 2*(D - 1)*XYH_21 + 2*(D^2 - 4*D + 3)*XYH_33 - (D^2 - 3*D)*XYV2 - (D^2 - 3*D + 2)*XYV3)*rho_2 + (2*(D - 2)*rho_2^2 + 2*(D - 2)*X0_22 + 4*(D - 1)*XH_12 + 2*(D^2 - 4*D + 3)*XH_33 - (D^2 - 3*D)*XV2 - (D^2 - 3*D + 2)*XV3)*rho_4 + 4*(((D - 1)*XYH_13 + (D - 1)*XYH_23 + (D - 1)*XYH_31 + (D - 1)*XYH_32)*rho_2 + ((D - 1)*XH_13 + (D - 1)*XH_23)*rho_4)*sqrt(D - 2))*sqrt(D - 1))/(D^2 - D)

In [10]:
## Expectation value <M_ij M_jk N_ki>
Npoint([[1,2],[3,4]],[[5,6]],[[1,6],[2,3],[4,5]]).full_simplify()

1/2*(2*(2*(D - 1)*XY0_11 + (D^2 - 2*D + 1)*XYH_12 + (D^2 - 2*D + 1)*XYH_21)*rho_1 + 2*((D - 1)*rho_1^2 + (D - 1)*X0_11 + (D^2 - 2*D + 1)*XH_12)*rho_3 + (2*((D - 1)*XYH_12 + (D - 1)*XYH_21 + 2*(D - 1)*XYH_33 + (D^2 - 3*D)*XYV2 - (D^2 - 3*D + 2)*XYV3 + 2*XY0_22)*rho_2 + (2*(D - 1)*XH_12 + 2*(D - 1)*XH_33 + (D^2 - 3*D)*XV2 - (D^2 - 3*D + 2)*XV3 + 2*rho_2^2 + 2*X0_22)*rho_4)*sqrt(D - 1))/(D - 1)

In [9]:
## Expectation value <M_ij M_kl N_pq>
Npoint([[1,2],[3,4]],[[5,6]],[[1],[2],[3],[4],[5],[6]]).full_simplify()

2*D^3*XY0_11*rho_1 + (D^3*rho_1^2 + D^3*X0_11)*rho_3

In [ ]:
## Quartic expectation values

In [15]:
## Expectation value <M_ij M_kl N_pq N_rs>
Npoint([[1,2],[3,4]],[[5,6],[7,8]],[[1],[2],[3],[4],[5],[6],[7],[8]]).full_simplify()

D^4*Y0_11*rho_1^2 + 4*D^4*XY0_11*rho_1*rho_3 + 2*D^4*XY0_11^2 + D^4*X0_11*Y0_11 + (D^4*rho_1^2 + D^4*X0_11)*rho_3^2

In [16]:
## Expectation value <M_ij M_kl N_iq N_rs>
Npoint([[1,2],[3,4]],[[5,6],[7,8]],[[1,5],[2],[3],[4],[6],[7],[8]]).full_simplify()

D^3*Y0_11*rho_1^2 + 2*D^3*XY0_11^2 + D^3*X0_11*Y0_11 + (D^4 - D^3)*XY0_11*XYH_22 + (4*D^3*XY0_11 + (D^4 - D^3)*XYH_22)*rho_1*rho_3 + (D^3*rho_1^2 + D^3*X0_11)*rho_3^2

In [17]:
## Expectation value <M_ij M_jk N_il N_lp>
Npoint([[1,2],[3,4]],[[5,6],[7,8]],[[1,5],[2,3],[4],[6],[7],[8]]).full_simplify()

D^2*Y0_11*rho_1^2 + 2*D^2*XY0_11^2 + (D^3 - D^2)*XY0_11*XYH_22 + (4*D^2*XY0_11 + (D^3 - D^2)*XYH_22)*rho_1*rho_3 + (D^2*rho_1^2 + D^2*X0_11 + (D^3 - D^2)*XH_12)*rho_3^2 + (D^2*X0_11 + (D^3 - D^2)*XH_12)*Y0_11 + (D^2*XYH_22*rho_2*rho_3 + D^2*XY0_21*XYH_22)*sqrt(D - 1)

In [18]:
## Expectation value <M_ij M_jk N_kl N_lp>
Npoint([[1,2],[3,4]],[[5,6],[7,8]],[[1],[2,3],[4,5],[6,7],[8]]).full_simplify()

D*XYH_12*rho_2*rho_4 + 2*D*XY0_11^2 + (D^2 - D)*XYH_12*XYH_21 + (D^2 - D)*XYH_11*XYH_22 + (D^2 - D)*XYH_13*XYH_32 + (D^2 - D)*XYH_12*XYH_33 + 1/2*(D^3 - 3*D^2)*XYH_12*XYV2 - 1/2*(D^3 - 3*D^2 + 2*D)*XYH_12*XYV3 + (D*Y0_11 + (D^2 - D)*YH_12)*rho_1^2 + (4*D*XY0_11 + (D^2 - D)*XYH_12)*rho_1*rho_3 + (D*rho_1^2 + D*X0_11 + (D^2 - D)*XH_12)*rho_3^2 + ((D^2 - D)*XY0_11 + D*XY0_22)*XYH_12 + (D*X0_11 + (D^2 - D)*XH_12)*Y0_11 + ((D^2 - D)*X0_11 + (D^3 - 2*D^2 + D)*XH_12)*YH_12 + (D*XYH_12*rho_2*rho_3 + D*XYH_12*rho_1*rho_4 + (D*XY0_12 + D*XY0_21)*XYH_12)*sqrt(D - 1)